In [4]:
#require_relative 'lib/openstudio/bem_to_surrogate/config.rb'
IRuby::Kernel.instance.switch_backend!(:pry)
$LOAD_PATH << File.expand_path('./lib') # Resolve global paths

require 'json'
require 'pathname'
require 'yaml'
require 'rexml'

require 'openstudio_measure_tester'

true

In [149]:
module JUnitType
  MINITEST = 'minitest'
  PYTEST = 'pytest'
  ACCEPTED_VALUES = [MINITEST, PYTEST]

  def self.validate(v)
    raise "Unknow value #{v}: #{ACCEPTED_VALUES}" unless ACCEPTED_VALUES.include?(v)
  end
end


def parse_junit_xunit1(xml_path, junit_type:)

  JUnitType.validate(junit_type)

  doc = REXML::Document.new(File.open(xml_path)).root

  if not doc
    return nil
  end

  testsuite_element = doc.elements['testsuite']
  errors, failures, skipped, annotations = parse_testsuite(testsuite_element, junit_type: junit_type)

  result = {
    #measure_compatibility_errors: json_data[:compatible] ? 0 1
   measure_tests: testsuite_element.attributes['tests'].to_i,
   measure_assertions: testsuite_element.attributes['assertions'].to_i,
   measure_errors: testsuite_element.attributes['errors'].to_i,
   measure_failures: testsuite_element.attributes['failures'].to_i,
   measure_skipped: testsuite_element.attributes['skipped'].to_i,
   issues: {
     errors: errors,
     failures: failures,
     skipped: skipped,
      # compatibility_error: json_data[:compatible] ? 0 1
    },
    annotations: annotations,
  }

  result
end

def parse_testsuite(testsuite_element, junit_type:)
  errors = []
  failures = []
  skipped = []

  JUnitType.validate(junit_type)

  annotations = []
  filepath = nil
  if junit_type == JUnitType::MINITEST
    filepath = testsuite_element.attributes['filepath']
  end

  testsuite_element.elements.each('testcase') do |testcase|
    if testcase.elements['error']
      errors << testcase.elements['error']
    elsif testcase.elements['failure']
      failures << testcase.elements['failure']
    elsif testcase.elements['skipped']
      skipped << 'Skipped test: ' + testcase.elements['skipped'].attributes['type']
    end

    annotations += prepare_testcase_annotations(testcase, junit_type: junit_type, filepath: filepath)
  end

  return errors, failures, skipped, annotations
end

def prepare_testcase_annotations(testcase, junit_type:, filepath: nil)

  annotations = []

  JUnitType.validate(junit_type)
  is_minitest = (junit_type == JUnitType::MINITEST)
  line_key = is_minitest ? 'lineno' : 'line'

  raise "When Minitest, filepath can't be nil" if is_minitest && filepath.nil?


  name = testcase.attributes['name']
  line = testcase.attributes[line_key].to_i
  classname = testcase.attributes['classname']

  # annot = "::error file=#{filepath},line=#{line},endLine=#{line + 1},title=#{title}::#{classname}.#{name}:#{message}"
  testcase.elements.each('failure') do |x|
    title = x.attributes['type']
    message = x.attributes['message']
    if !is_minitest
      filepath = testcase.attributes['file']
    end
    annotations << {
      type: 'error',
      classname: classname,
      name: name,
      file: filepath,
      line: line,
      title: title,
      message: message,
    }
  end
  testcase.elements.each('error') do |x|
    title = x.attributes['type']
    message = x.attributes['message']
    if !is_minitest
      filepath = testcase.attributes['file']
    end
    annotations << {
      type: 'error',
      classname: classname,
      name: name,
      file: filepath,
      line: line,
      title: title,
      message: message,
    }
  end
  testcase.elements.each('skipped') do |x|
    title = x.attributes['type']
    message = x.attributes['message']
    if !is_minitest
      filepath = testcase.attributes['file']
    end
    annotations << {
      type: 'warning',
      classname: classname,
      name: name,
      file: filepath,
      line: line,
      title: title,
      message: message,
    }
  end

  return annotations
end

def format_github_annotation(annotation_hash)
  "::#{annotation_hash[:type]} file=#{annotation_hash[:file]},line=#{annotation_hash[:line]},endLine=#{annotation_hash[:line] + 1},title=#{annotation_hash[:title]}::#{annotation_hash[:classname]}.#{annotation_hash[:name]}:#{annotation_hash[:message]}"
end

(irb):1: warning: already initialized constant #<Class:0x0000556432993ec8>::JUnitType::MINITEST
(irb):1: warning: previous definition of MINITEST was here
(irb):2: warning: already initialized constant #<Class:0x0000556432993ec8>::JUnitType::PYTEST
(irb):2: warning: previous definition of PYTEST was here
(irb):3: warning: already initialized constant #<Class:0x0000556432993ec8>::JUnitType::ACCEPTED_VALUES
(irb):3: warning: previous definition of ACCEPTED_VALUES was here


:format_github_annotation

In [150]:
path_to_results = 'spec/files/pytest'
@path_to_results = Pathname.new(path_to_results)
@junit_xml = @path_to_results / 'junit.xml'
@has_results = @path_to_results.directory?
@has_results ||= @junit_xml.file?

true

In [158]:
h = parse_junit_xunit1(@junit_xml, junit_type: JUnitType::PYTEST)
puts format_github_annotation(h[:annotations][0])

::error file=python_modelmeasure/tests/test_my_example_python_model_measure.py,line=18,endLine=19,title=::python_modelmeasure.tests.test_my_example_python_model_measure.TestMyExamplePythonModelMeasure.test_number_of_arguments_and_argument_names:assert 1 == 2


In [159]:
h[:annotations][0]

{:type=>"error", :classname=>"python_modelmeasure.tests.test_my_example_python_model_measure.TestMyExamplePythonModelMeasure", :name=>"test_number_of_arguments_and_argument_names", :file=>"python_modelmeasure/tests/test_my_example_python_model_measure.py", :line=>18, :title=>nil, :message=>"assert 1 == 2"}

In [153]:
h = parse_junit_xunit1('spec/files/minitest/reports/TEST-AddOverhangsByProjectionFactor-Test.xml',
  junit_type: JUnitType::MINITEST)
puts format_github_annotation(h[:annotations][0])

::error file=spec/test_measures/AddOverhangsByProjectionFactor/tests/AddOverhangsByProjectionFactor_Test.rb,line=192,endLine=193,title=test_failure::AddOverhangsByProjectionFactor_Test.test_failure:RuntimeError: This will be an error...


In [ ]:
format_github_annotation

In [41]:
  result = {}
  # result[:measure_compatibility_errors] = json_data[:compatible] ? 0 : 1
  result[:measure_tests] = 0
  result[:measure_assertions] = 0
  result[:measure_errors] = 0
  result[:measure_failures] = 0
  result[:measure_skipped] = 0
  result[:issues] = {
    errors: [],
    failures: [],
    skipped: [],
    # compatibility_error: json_data[:compatible] ? 0 : 1
  }
puts JSON.pretty_generate result

{
  "measure_tests": 0,
  "measure_assertions": 0,
  "measure_errors": 0,
  "measure_failures": 0,
  "measure_skipped": 0,
  "issues": {
    "errors": [

    ],
    "failures": [

    ],
    "skipped": [

    ]
  }
}


In [16]:
doc = REXML::Document.new(@junit_xml.read).root

<testsuites> ... </>

In [49]:
puts doc

<testsuites>
  <testsuite errors='0' failures='1' hostname='julien-desktop' name='pytest' skipped='0' tests='6' time='0.826' timestamp='2024-04-17T16:07:36.989631'>
    <testcase classname='python_energyplusmeasure.tests.test_my_example_python_energyplus_measure.TestMyExamplePythonEnergyPlusMeasure' file='test_python_energyplusmeasure/tests/test_my_example_python_energyplus_measure.py' line='18' name='test_number_of_arguments_and_argument_names' time='0.003'/>
    <testcase classname='python_energyplusmeasure.tests.test_my_example_python_energyplus_measure.TestMyExamplePythonEnergyPlusMeasure' file='test_python_energyplusmeasure/tests/test_my_example_python_energyplus_measure.py' line='33' name='test_bad_argument_values' time='0.004'/>
    <testcase classname='python_energyplusmeasure.tests.test_my_example_python_energyplus_measure.TestMyExamplePythonEnergyPlusMeasure' file='test_python_energyplusmeasure/tests/test_my_example_python_energyplus_measure.py' line='62' name='test_good_argu

In [116]:
x = nil
all_annotations = []
testsuite_element.elements.each('testcase') do |testcase|
  # ["classname", "name", "file", "line", "time"]
  classname = testcase.attributes['classname']
  name = testcase.attributes['name']
  filepath = testcase.attributes['file']
  line = testcase.attributes['line'].to_i
  testcase.elements.each('failure') do |x|
    title = x.attributes['message']
    message = x.text
    
    all_annotations << {
      classname: classname,
      name: name,
      file: filepath,
      line: line,
      title: title,
      # message: "#{title}\n\n#{message}",
      message: message,
    }
  end
end

[<testcase classname='python_energyplusmeasure.tests.test_my_example_python_energyplus_measure.TestMyExamplePythonEnergyPlusMeasure' name='test_number_of_arguments_and_argument_names' file='test_python_energyplusmeasure/tests/test_my_example_python_energyplus_measure.py' line='18' time='0.003'/>, <testcase classname='python_energyplusmeasure.tests.test_my_example_python_energyplus_measure.TestMyExamplePythonEnergyPlusMeasure' name='test_bad_argument_values' file='test_python_energyplusmeasure/tests/test_my_example_python_energyplus_measure.py' line='33' time='0.004'/>, <testcase classname='python_energyplusmeasure.tests.test_my_example_python_energyplus_measure.TestMyExamplePythonEnergyPlusMeasure' name='test_good_argument_values' file='test_python_energyplusmeasure/tests/test_my_example_python_energyplus_measure.py' line='62' time='0.003'/>, <testcase classname='python_modelmeasure.tests.test_my_example_python_model_measure.TestMyExamplePythonModelMeasure' name='test_number_of_argumen

In [117]:
puts JSON.pretty_generate all_annotations[0]

{
  "classname": "python_modelmeasure.tests.test_my_example_python_model_measure.TestMyExamplePythonModelMeasure",
  "name": "test_number_of_arguments_and_argument_names",
  "file": "python_modelmeasure/tests/test_my_example_python_model_measure.py",
  "line": 18,
  "title": "assert 1 == 2",
  "message": "self = <test_my_example_python_model_measure.TestMyExamplePythonModelMeasure object at 0x7fc9c1145cd0>\n\n    def test_number_of_arguments_and_argument_names(self):\n        \"\"\"Test that the arguments are what we expect.\"\"\"\n        # create an instance of the measure\n        measure = MyExamplePythonModelMeasure()\n    \n        # make an empty model\n        model = openstudio.model.Model()\n    \n        # get arguments and test that they are what we are expecting\n        arguments = measure.arguments(model)\n        num_args = arguments.size()\n>       assert num_args == 2\nE       assert 1 == 2\n\npython_modelmeasure/tests/test_my_example_python_model_measure.py:30: Asser

In [91]:
puts all_annotations[0][:message]

assert 1 == 2

self = <test_my_example_python_model_measure.TestMyExamplePythonModelMeasure object at 0x7fc9c1145cd0>

    def test_number_of_arguments_and_argument_names(self):
        """Test that the arguments are what we expect."""
        # create an instance of the measure
        measure = MyExamplePythonModelMeasure()
    
        # make an empty model
        model = openstudio.model.Model()
    
        # get arguments and test that they are what we are expecting
        arguments = measure.arguments(model)
        num_args = arguments.size()
>       assert num_args == 2
E       assert 1 == 2

python_modelmeasure/tests/test_my_example_python_model_measure.py:30: AssertionError


In [80]:
testcase.attributes.keys

["classname", "name", "file", "line", "time"]

"self = <test_my_example_python_model_measure.TestMyExamplePythonModelMeasure object at 0x7fc9c1145cd0>\n\n    def test_number_of_arguments_and_argument_names(self):\n        \"\"\"Test that the arguments are what we expect.\"\"\"\n        # create an instance of the measure\n        measure = MyExamplePythonModelMeasure()\n    \n        # make an empty model\n        model = openstudio.model.Model()\n    \n        # get arguments and test that they are what we are expecting\n        arguments = measure.arguments(model)\n        num_args = arguments.size()\n>       assert num_args == 2\nE       assert 1 == 2\n\npython_modelmeasure/tests/test_my_example_python_model_measure.py:30: AssertionError"

In [96]:
enum {
  Minitest,
  BAR,
  BAZ
}

SyntaxError: (irb):3: syntax error, unexpected '\n', expecting '='

In [99]:
module JUnitType
  MINITEST = 1
  PYTEST = 2
end

2

In [111]:
module JUnitType
  MINITEST = 'minitest'
  PYTEST = 'pytest'
  ACCEPTED_VALUES = [MINITEST, PYTEST]
  
  def self.validate(v)
    raise "Unknow value #{v}: #{ACCEPTED_VALUES}" unless ACCEPTED_VALUES.include?(v)
  end
end

(irb):1: warning: already initialized constant #<Class:0x0000556432993ec8>::JUnitType::MINITEST
(irb):1: warning: previous definition of MINITEST was here
(irb):2: warning: already initialized constant #<Class:0x0000556432993ec8>::JUnitType::PYTEST
(irb):2: warning: previous definition of PYTEST was here


:validate

In [115]:
JUnitType.validate("caca")

RuntimeError: Unknow value caca: ["minitest", "pytest"]